In [15]:
from numpy import nan
from numpy import isnan
from pandas import to_numeric
from numpy import split
from numpy import array

In [3]:
import pandas as pd
# load all data
dataset = pd.read_csv('household_power_consumption.txt', sep=';', header=0, low_memory=False, infer_datetime_format=True, parse_dates={'datetime':[0,1]}, index_col=['datetime'])

In [6]:
# fill missing values with a value at the same time one day ago
def fill_missing(values):
	one_day = 60 * 24
	for row in range(values.shape[0]):
		for col in range(values.shape[1]):
			if isnan(values[row, col]):
				values[row, col] = values[row - one_day, col]

In [8]:
# mark all missing values
dataset.replace('?', nan, inplace=True)
# make dataset numeric (数值化)
dataset = dataset.astype('float32') # 将数据集的所有数据转化为ｆｌａｏｔ３２类型
# fill missing
fill_missing(dataset.values)
# add a column for for the remainder of sub metering
values = dataset.values
dataset['sub_metering_4'] = (values[:,0] * 1000 / 60) - (values[:,4] + values[:,5] + values[:,6])
# save updated dataset
dataset.to_csv('household_power_consumption.csv')

In [9]:
# resample minute data to total for each day
from pandas import read_csv
# load the new file
dataset = read_csv('household_power_consumption.csv', header=0, infer_datetime_format=True, parse_dates=['datetime'], index_col=['datetime'])
# resample data to daily
daily_groups = dataset.resample('D')
daily_data = daily_groups.sum()
# summarize
print(daily_data.shape)
print(daily_data.head())
# save
daily_data.to_csv('household_power_consumption_days.csv')

(1442, 8)
            Global_active_power  Global_reactive_power        Voltage  \
datetime                                                                
2006-12-16          1209.175999                 34.922   93552.529953   
2006-12-17          3390.460002                226.006  345725.320053   
2006-12-18          2203.826000                161.792  347373.640137   
2006-12-19          1666.194001                150.942  348479.009842   
2006-12-20          2225.748000                160.998  348923.610077   

            Global_intensity  Sub_metering_1  Sub_metering_2  Sub_metering_3  \
datetime                                                                       
2006-12-16       5180.800003             0.0           546.0          4926.0   
2006-12-17      14398.600012          2033.0          4187.0         13341.0   
2006-12-18       9247.199997          1063.0          2621.0         14018.0   
2006-12-19       7094.000005           839.0          7602.0          6197.0  

In [85]:
'''
dataset这个数据集是一个1442*8的数组
而dataset经过split_dataset函数后被分成了再次划分成了7份(split函数从左到右划分，故先将第一维的1442划分成了7份，而不是对第二维的8划分)，
划分之后我们旧得到了train数据集，后面出现的数据扁平化(flatten data)实际上就是将train的结构还原为dataset的结构.
train.shape为(159, 7, 8)
test.shape为(46, 7, 8)
train训练集是一个三维的结构
三个维度分别代表了:batch的数量，滑动窗口的大小，每一个样例的特征数量
batch是训练
'''

'\ndataset这个数据集是一个1442*8的数组\n而dataset经过split_dataset函数后被分成了再次划分成了7份(split函数从左到右划分，故先将第一维的1442划分成了7份，而不是对第二维的8划分)，\n划分之后我们旧得到了train数据集，后面出现的数据扁平化(flatten data)实际上就是将train的结构还原为dataset的结构.\ntrain.shape为(159, 7, 8)\ntest.shape为(46, 7, 8)\ntrain训练集是一个三维的结构\n三个维度分别代表了:batch的数量，滑动窗口的大小，每一个样例的特征数量\nbatch是训练\n'

In [86]:
from math import sqrt
from numpy import split
from numpy import array
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import LSTM
 
# split a univariate dataset into train/test sets
# 训练集划分，将单变量数据集划分为池训练集和测试集
# 并且还有一点值得说明，训练LSTM的时候我们是以一天为step的训练的，而预测的时候是一周为step进行预测的，这也就是为什么要按周split数据集的原因
def split_dataset(data):
	# 讲前三年的数据作为训练集，将最后一年的数据作为测试集
	train, test = data[1:-328], data[-328:-6]
	# 将训练数据重组为以周为单位的数据
    # split函数是一个numpy库的函数，其作用是把一个array从左到右按顺序切分，其
    # 切分长度不能超过array的元素个数,axis默认为０，即横向切分
	train = array(split(train, len(train)/7))
	test = array(split(test, len(test)/7))
	return train, test
 
# evaluate one or more weekly forecasts against expected values
# 根据预期值评价单周预测或者多周预测
def evaluate_forecasts(actual, predicted):
    # 参数说明：actual是实际值，predicted是预测值
	scores = list()
	# calculate an RMSE score for each day
    # 为每一日的预测值计算ＲＭＳＥ(均方根误差)评分
	for i in range(actual.shape[1]):
		# 计算平方误差
		mse = mean_squared_error(actual[:, i], predicted[:, i])
		# 计算均方根误差
		rmse = sqrt(mse)
		# 储存到scores容器中
		scores.append(rmse)
	# 计算全体全体测试集和预测值的均方根误差
	s = 0
	for row in range(actual.shape[0]):
		for col in range(actual.shape[1]):
			s += (actual[row, col] - predicted[row, col])**2
	score = sqrt(s / (actual.shape[0] * actual.shape[1]))
	return score, scores
 
# 计算得分的总和
def summarize_scores(name, score, scores):
    # join函数(python系统自带函数)是将列(list)表转化为字符串的函数，单引号中的逗号是分隔符。
	s_scores = ', '.join(['%.1f' % s for s in scores])
	print('%s: [%.3f] %s' % (name, score, s_scores))
    
# convert history into inputs and outputs
def to_supervised(train, n_input, n_out=7):
    # 参数说明:n_input是滑动窗口大小,n_out是未来预测的步长，默认为7即说明我们要预测未来7天，即一周，的数据。
	# flatten data,数据扁平化
	data = train.reshape((train.shape[0]*train.shape[1], train.shape[2]))
	X, y = list(), list()
	in_start = 0
	# step over the entire history one time step at a time
	for _ in range(len(data)):
		# define the end of the input sequence
		in_end = in_start + n_input  # 预测的输入窗口截止索引(输入窗口大小：in_end-in_start=7)
		out_end = in_end + n_out     # 预测的输出窗口截止索引(输出窗口大小：out_end-in_end=7)
		# ensure we have enough data for this instance，保证输出窗口的移动不会超过数据集的边界
		if out_end < len(data):
			x_input = data[in_start:in_end, 0]  # 由于是单特征预测，所以这里只取一个特征,x_input的结构是[1,2,...,8]这样的结构
            # 下面rashape的目的是将输出数据x_input转化为2d的形式，即[[1],[2],[3],..,[8]]的形式，这个是为了满足keras模型的输入
			x_input = x_input.reshape((len(x_input), 1))   # 这里要注意len()一个多维数组返回的是其最外层的维度大小
			X.append(x_input)
			y.append(data[in_end:out_end, 0])  # 标签y无需转化为2D形式
		# move along one time step
		in_start += 1
	return array(X), array(y)
 
# train the model
def build_model(train, n_input):
	# prepare data,将时间序列数据转化为符合监督学习的格式
	train_x, train_y = to_supervised(train, n_input)
	# define parameters，确定参数
	verbose, epochs, batch_size = 0, 70, 16
	n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]
	# define model，定义模型结构
	model = Sequential()
	model.add(LSTM(200, activation='relu', input_shape=(n_timesteps, n_features)))
	model.add(Dense(100, activation='relu'))
	model.add(Dense(n_outputs))
	model.compile(loss='mse', optimizer='adam')
	# fit network，拟合网络
	model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, verbose=verbose)
	return model

'''

python不允许程序员选择采用传值还是传引用。
Python参数传递采用的肯定是“传对象引用”的方式。
这种方式相当于传值和传引用的一种综合。
如果函数收到的是一个可变对象（比如字典或者列表）的引用，
就能修改对象的原始值－－相当于通过“传引用”来传递对象。
如果函数收到的是一个不可变对象（比如数字、字符或者元组）的引用，
就不能直接修改原始对象－－相当于通过“传值'来传递对象。

'''
# make a forecast，进行一次预测
'''
forecast函数的预测规则：
n_input是滑动窗口的大小，即我们每次用最后n_input个周的历史数据去预测下一个周的数据，这个“历史数据”就来自
evaluate_model中history集合，即每次都用离待预测数据最近的n_input个连续数据去预测接下来最近时刻的情况，
这样充分利用了数据之间的时序信息，体现了时间序列模型与其他回归模型在实现上的不同。
'''
def forecast(model, history, n_input):
	# flatten data，数据扁平化
	data = array(history)
	data = data.reshape((data.shape[0]*data.shape[1], data.shape[2]))
	# retrieve last observations for input data，从输入数据中提取最近的观测值
	input_x = data[-n_input:, 0]
	# reshape into [1, n_input, 1]，讲数据变换成符合lstm模型的输入格式
	input_x = input_x.reshape((1, len(input_x), 1))
	# forecast the next week,预测下一周的数据
	yhat = model.predict(input_x, verbose=0)
	# we only want the vector forecast，这个地方不太清楚，为什么只取第一项，应该和model.predict的返回值有关
	yhat = yhat[0]
	return yhat
 
# evaluate a single model
# 使用的是前移评价(Walk Forward Validation)方法(时间序列模型中的k折交叉验证)
'''
evaluate_model中history数据集合的作用和更新规则:
在最初history是等于训练集train的，随后，在每一轮的预测中，每取出一个测试集的样例，在预测函数forcast调用结束
之后就将其加入到history集合中,最后history=train+test

'''
def evaluate_model(train, test, n_input):
	# fit model
	model = build_model(train, n_input)
	# history is a list of weekly data
    # 注意这里为什么不写成history=train,因为python中只有引用，没有赋值,所以必须将train"复制"一份才可以赋值给history
	history = [x for x in train]
	# walk-forward validation over each week，对每一次预测都进行前移评价
	predictions = list()
	for i in range(len(test)):
		# predict the week,得到一个test样例的预测结果
		yhat_sequence = forecast(model, history, n_input)
		# store the predictions，储存预测结果
		predictions.append(yhat_sequence)
		# get real observation and add to history for predicting the next week，讲该test样例当做历史数据加入到history数据集中作为下一次预测的输入
		history.append(test[i, :])
	# evaluate predictions days for each week，对每一周的预测结果进行评价
	predictions = array(predictions)
	score, scores = evaluate_forecasts(test[:, :, 0], predictions)
	return score, scores
 
# load the new file
dataset = read_csv('household_power_consumption_days.csv', header=0, infer_datetime_format=True, parse_dates=['datetime'], index_col=['datetime'])
# split into train and test
train, test = split_dataset(dataset.values)
# evaluate model and get scores
n_input = 7
score, scores = evaluate_model(train, test, n_input)
# summarize scores
summarize_scores('lstm', score, scores)
# plot scores
days = ['sun', 'mon', 'tue', 'wed', 'thr', 'fri', 'sat']
pyplot.plot(days, scores, marker='o', label='lstm')
pyplot.show()

Using TensorFlow backend.


lstm: [397.186] 409.4, 407.7, 375.2, 391.2, 400.4, 328.9, 456.3


ValueError: could not convert string to float: 'sun'